In [1]:
import os
import sys

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Required for Spark to find Python executable
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder.appName("jira_extract_issues").getOrCreate()

spark.version

'3.5.3'

In [3]:
DATA_OUTPUT_DIR = "../data"

In [5]:
estabelecimentos_folder = f"{DATA_OUTPUT_DIR}/Estabelecimentos/"
files = os.listdir(estabelecimentos_folder)

files_to_transform = [f"{estabelecimentos_folder}{f}" for f in files] # if not f.endswith(".csv")]

files_to_transform

['../data/Estabelecimentos/K3241.K03200Y0.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y1.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y2.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y3.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y4.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y5.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y6.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y7.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y8.D41109.ESTABELE',
 '../data/Estabelecimentos/K3241.K03200Y9.D41109.ESTABELE']

In [6]:
csv_estabelecimentos_schema = StructType([
    StructField("cnpj_basico", StringType(), nullable=True),
    StructField("cnpj_ordem", StringType(), nullable=True),
    StructField("cnpj_dv", StringType(), nullable=True),
    StructField("cd_matriz_filial", IntegerType(), nullable=True),
    StructField("nome_fantasia", StringType(), nullable=True),
    StructField("cd_situacao_cadastral", StringType(), nullable=True),
    StructField("data_situacao_cadastral", StringType(), nullable=True),
    StructField("cd_motivo_situacao_cadastral", StringType(), nullable=True),
    StructField("nome_cidade_exterior", StringType(), nullable=True),
    StructField("pais", StringType(), nullable=True),
    StructField("data_inicio_atividades", StringType(), nullable=True),
    StructField("cd_cnae_fiscal", StringType(), nullable=True),
    StructField("cd_cnae_fiscal_secundario", StringType(), nullable=True),
    StructField("tipo_logradouro", StringType(), nullable=True),
    StructField("logradouro", StringType(), nullable=True),
    StructField("numero", IntegerType(), nullable=True),
    StructField("complemento", StringType(), nullable=True),
    StructField("bairro", StringType(), nullable=True),
    StructField("cep", StringType(), nullable=True),
    StructField("uf", StringType(), nullable=True),
    StructField("municipio", StringType(), nullable=True),
    StructField("ddd1", StringType(), nullable=True),
    StructField("telefone1", StringType(), nullable=True),
    StructField("ddd2", StringType(), nullable=True),
    StructField("telefone2", StringType(), nullable=True),
    StructField("ddd_fax", StringType(), nullable=True),
    StructField("fax", StringType(), nullable=True),
    StructField("correio_eletronico", StringType(), nullable=True),
    StructField("cd_situacao_especial", StringType(), nullable=True),
    StructField("data_situacao_especial", StringType(), nullable=True),
])

In [7]:
df_estabelecimentos = spark.read.csv(files_to_transform, header=False, sep=";", schema=csv_estabelecimentos_schema)

In [8]:
df_estabelecimentos.count()

63333645

In [118]:
df_estabelecimentos.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- cd_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cd_situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- cd_motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_inicio_atividades: string (nullable = true)
 |-- cd_cnae_fiscal: string (nullable = true)
 |-- cd_cnae_fiscal_secundario: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- tele

In [119]:
df_estabelecimentos.show()

+-----------+----------+-------+----------------+--------------------+---------------------+-----------------------+----------------------------+--------------------+----+----------------------+--------------+-------------------------+---------------+--------------------+------+--------------------+--------------------+--------+---+---------+----+---------+----+---------+-------+----+--------------------+--------------------+----------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|cd_matriz_filial|       nome_fantasia|cd_situacao_cadastral|data_situacao_cadastral|cd_motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividades|cd_cnae_fiscal|cd_cnae_fiscal_secundario|tipo_logradouro|          logradouro|numero|         complemento|              bairro|     cep| uf|municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax| fax|  correio_eletronico|cd_situacao_especial|data_situacao_especial|
+-----------+----------+-------+----------------+--------------------+---------------------+--

In [11]:
df_estab_tratado = (
    df_estabelecimentos
    .withColumn("cnpj", F.concat("cnpj_basico", "cnpj_ordem", "cnpj_dv"))
    .withColumn("matriz_filial", F.when(F.col("cd_matriz_filial") == 1, "Matriz").otherwise("Filial"))
    .withColumn("situacao_cadastral",
                F.when(F.col("cd_situacao_cadastral") == "01", "Nula")
                .when(F.col("cd_situacao_cadastral") == "02", "Ativa")
                .when(F.col("cd_situacao_cadastral") == "03", "Suspensa")
                .when(F.col("cd_situacao_cadastral") == "04", "Inapta")
                .otherwise("Baixada"))
    .withColumn("data_situacao_cadastral", F.to_date("data_situacao_cadastral", 'yyyyMMdd'))
    .withColumn("data_inicio_atividades", F.to_date("data_inicio_atividades", 'yyyyMMdd'))
    .withColumn("data_situacao_especial", F.to_date("data_situacao_especial", 'yyyyMMdd'))
)

In [138]:
df_estab_tratado.show(truncate=False)

+-----------+
|cnpj_basico|
+-----------+
|45855495   |
|45855505   |
|45855516   |
|45855528   |
|45855539   |
|45855550   |
|45855560   |
|45855570   |
|43909127   |
|45855595   |
|45855604   |
|45855615   |
|45855628   |
|45855639   |
|45855649   |
|45855661   |
|45855672   |
|45855684   |
|45855694   |
|45855706   |
+-----------+
only showing top 20 rows



In [12]:
motivos_folder = f"{DATA_OUTPUT_DIR}/Motivos/"
files = os.listdir(motivos_folder)

files_motivos_to_transform = [f"{motivos_folder}{f}" for f in files]

files_motivos_to_transform

['../data/Motivos/F.K03200$Z.D41109.MOTICSV']

In [13]:
csv_code_description_schema = StructType([
    StructField("codigo", StringType(), nullable=True),
    StructField("descricao", StringType(), nullable=True)
])

In [14]:
df_motivos = spark.read.csv(files_motivos_to_transform, header=False, sep=";", schema=csv_code_description_schema)

In [15]:
df_motivos.show(truncate=False)

+------+-------------------------------------------------+
|codigo|descricao                                        |
+------+-------------------------------------------------+
|00    |SEM MOTIVO                                       |
|01    |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA  |
|02    |INCORPORACAO                                     |
|03    |FUSAO                                            |
|04    |CISAO TOTAL                                      |
|05    |ENCERRAMENTO DA FALENCIA                         |
|06    |ENCERRAMENTO DA LIQUIDACAO                       |
|07    |ELEVACAO A MATRIZ                                |
|08    |TRANSPASSE                                       |
|09    |NAO INICIO DE ATIVIDADE                          |
|10    |EXTINCAO PELO ENCERRAMENTO DA LIQUIDACAO JUDICIAL|
|11    |ANULACAO POR MULTICIPLIDADE                      |
|12    |ANULACAO ONLINE DE OFICIO                        |
|13    |OMISSA CONTUMAZ                                 

In [16]:
df_estab_motivos = (
    df_estab_tratado
    .join(df_motivos,
          df_estab_tratado.cd_motivo_situacao_cadastral == df_motivos.codigo, "left")
    .select(
        df_estab_tratado["*"],  # Todas as colunas do df_estab_tratado
        df_motivos["descricao"].alias("motivo_situacao_cadastral")  # Apenas a coluna "descricao" do df_motivos
    ))

In [127]:
df_estab_motivos.show()

+-----------+----------+-------+----------------+--------------------+---------------------+-----------------------+----------------------------+--------------------+----+----------------------+--------------+-------------------------+---------------+--------------------+------+--------------------+--------------------+--------+---+---------+----+---------+----+---------+-------+----+--------------------+--------------------+----------------------+--------------+-------------+------------------+-------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|cd_matriz_filial|       nome_fantasia|cd_situacao_cadastral|data_situacao_cadastral|cd_motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividades|cd_cnae_fiscal|cd_cnae_fiscal_secundario|tipo_logradouro|          logradouro|numero|         complemento|              bairro|     cep| uf|municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax| fax|  correio_eletronico|cd_situacao_especial|data_situacao_especial|          cnpj|matriz

In [17]:
cnaes_folder = f"{DATA_OUTPUT_DIR}/Cnaes/"
files = os.listdir(cnaes_folder)

files_cnaes_to_transform = [f"{cnaes_folder}{f}" for f in files]

files_cnaes_to_transform

['../data/Cnaes/F.K03200$Z.D41109.CNAECSV']

In [18]:
df_cnaes = spark.read.csv(files_cnaes_to_transform, header=False, sep=";", schema=csv_code_description_schema,
                          encoding="ISO-8859-1")

In [19]:
df_cnaes.show(truncate=False)

+-------+-----------------------------------------------------------------------------------+
|codigo |descricao                                                                          |
+-------+-----------------------------------------------------------------------------------+
|0111301|Cultivo de arroz                                                                   |
|0111302|Cultivo de milho                                                                   |
|0111303|Cultivo de trigo                                                                   |
|0111399|Cultivo de outros cereais não especificados anteriormente                          |
|0112101|Cultivo de algodão herbáceo                                                        |
|0112102|Cultivo de juta                                                                    |
|0112199|Cultivo de outras fibras de lavoura temporária não especificadas anteriormente     |
|0113000|Cultivo de cana-de-açúcar                          

In [20]:
df_estab_motivos_cnaes = (
    df_estab_motivos
    .join(df_cnaes,
          df_estab_motivos.cd_cnae_fiscal == df_cnaes.codigo, "left")
    .select(
        df_estab_motivos["*"],  # Todas as colunas do df_estab_tratado
        df_cnaes["descricao"].alias("cnae_fiscal")  # Apenas a coluna "descricao" do df_motivos
    ))

In [132]:
df_estab_motivos_cnaes.show()

+-----------+----------+-------+----------------+--------------------+---------------------+-----------------------+----------------------------+--------------------+----+----------------------+--------------+-------------------------+---------------+--------------------+------+--------------------+--------------------+--------+---+---------+----+---------+----+---------+-------+----+--------------------+--------------------+----------------------+--------------+-------------+------------------+-------------------------+--------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|cd_matriz_filial|       nome_fantasia|cd_situacao_cadastral|data_situacao_cadastral|cd_motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividades|cd_cnae_fiscal|cd_cnae_fiscal_secundario|tipo_logradouro|          logradouro|numero|         complemento|              bairro|     cep| uf|municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax| fax|  correio_eletronico|cd_situacao_especial|data_situacao_especial|

In [21]:
# O Select apenas ajusta a ordem das colunas para que sejam persistidas 
# de uma maneira mais facil de visualizar, colocando as descricoes 
# ao lado dos codigos
df_estabelecimentos_final = df_estab_motivos_cnaes.select('cnpj_basico',
                                                          'cnpj_ordem',
                                                          'cnpj_dv',
                                                          'cnpj',
                                                          'cd_matriz_filial',
                                                          'matriz_filial',
                                                          'nome_fantasia',
                                                          'cd_situacao_cadastral',
                                                          'situacao_cadastral',
                                                          'data_situacao_cadastral',
                                                          'cd_motivo_situacao_cadastral',
                                                          'motivo_situacao_cadastral',
                                                          'nome_cidade_exterior',
                                                          'pais',
                                                          'data_inicio_atividades',
                                                          'cd_cnae_fiscal',
                                                          'cnae_fiscal',
                                                          'cd_cnae_fiscal_secundario',
                                                          'tipo_logradouro',
                                                          'logradouro',
                                                          'numero',
                                                          'complemento',
                                                          'bairro',
                                                          'cep',
                                                          'uf',
                                                          'municipio',
                                                          'ddd1',
                                                          'telefone1',
                                                          'ddd2',
                                                          'telefone2',
                                                          'ddd_fax',
                                                          'fax',
                                                          'correio_eletronico',
                                                          'cd_situacao_especial',
                                                          'data_situacao_especial')

In [23]:
df_estabelecimentos_final.show(truncate=False)

+-----------+----------+-------+--------------+----------------+-------------+----------------------------------+---------------------+------------------+-----------------------+----------------------------+-------------------------+--------------------+----+----------------------+--------------+----------------------------------------------------------------------------+-------------------------+---------------+----------+------+-----------+------+--------+---+---------+----+---------+----+---------+-------+----+--------------------+--------------------+----------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|cnpj          |cd_matriz_filial|matriz_filial|nome_fantasia                     |cd_situacao_cadastral|situacao_cadastral|data_situacao_cadastral|cd_motivo_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividades|cd_cnae_fiscal|cnae_fiscal                                                                 |cd_cnae_fiscal_secundario|tipo_logradouro